#### Point Map Visualization

Point map visualization is natively supported, if your table outputs longitude-latitude pairs. It is recommended to limit the query to not too many rows for performance reasons.

> [!NOTE]
> We are using Spatial Functions and the GEOMETRY data type below, and as of 2025-07-16, you need to use Databricks Runtime 17.1+ for this. It is expected that this feature becomes available on Serverless Compute and DBSQL as well soon, see [here](https://www.linkedin.com/feed/update/urn:li:activity:7351276934338641920/).

In [ ]:
%pip install geoarrow-pyarrow lonboard shapely --quiet

In [ ]:
from lonboard import viz
import geoarrow.pyarrow as ga
import pyarrow as pa
import pyproj

In [ ]:
%sql
create
or replace table tmp_geometries as
select
  "Onze Lieve Vrouwetoren" as name,
  st_transform(st_point(155000, 463000, 28992), 4326) as geometry
union all
select
  "Bermuda Triangle" as name,
  st_makepolygon(
    st_makeline(
      array(
        st_point(- 80.1935973, 25.7741566, 4326),
        st_point(- 64.7563086, 32.3040273, 4326),
        st_point(- 66.1166669, 18.4653003, 4326),
        st_point(- 80.1935973, 25.7741566, 4326)
      )
    )
  ) as geometry;

select
  *
from
  tmp_geometries

Our example table contains not only points, but the native visualization can only show points, so one thing we can do is to show the centroids.

In the above example, the SRID happens to be 4326, therefore the below `st_transform` is a unnecessary. In general, however, you'd want to transform the geometries to 4326 in order to obtain valid `(lon, lat)` tuples.

In [ ]:
%sql
select
  name,
  st_transform(st_centroid(geometry), 4326) centroid,
  st_x(centroid) lon,
  st_y(centroid) lat
from
  tmp_geometries

In [ ]:
dfa = spark.table("tmp_geometries").limit(10_000).toArrow()

In [ ]:
# Assuming that all SRID's within the table are the same, which they should be as the
# SRID is part of the column type.
crs = pyproj.CRS.from_epsg(dfa["geometry"][0]["srid"]).to_json()

In [ ]:
field_chunks = [chunk.field("wkb") for chunk in dfa["geometry"].chunks]
dfa_geoarrow = dfa.append_column(
    "geoarrow", ga.with_crs(ga.as_geoarrow(pa.chunked_array(field_chunks)), crs)
).drop("geometry")

In [ ]:
viz(dfa_geoarrow).as_html()

Alternatively, you can use DuckDB to visualize in lonboard, see the DuckDB tutorial for that.